Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [76]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [17]:
train_labels[:]

array([0, 1, 4, ..., 1, 8, 2], dtype=int32)

In [10]:
np.arange(len(train_labels)) == train_labels[:, None]

array([[ True, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False,  True, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False]], dtype=bool)

In [77]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [67]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [78]:
def multilayer_perceptron(x, weights, biases):
     # Training computation.
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1_output = tf.nn.relu(layer_1)
    ## change data to relu
    
    logits = tf.matmul(layer_1_output, weights['h2']) + biases['b2']
    
    return logits


def regularization(weights):
    alpha = 0.007
    return alpha * sum(map(lambda w_i: tf.nn.l2_loss(w_i), weights.values()))


batch_size = 128

    # initial neoral network by train size @param batch_size
n_hidden = 1024
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
    
        weights = {
            'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden])),
            'h2': tf.Variable(tf.truncated_normal([n_hidden, num_labels])),
            # output is num_labels
        }

        biases = {
            'b1': tf.Variable(tf.zeros([n_hidden])), # size equal to output nodess
            'b2': tf.Variable(tf.zeros([num_labels])), # size equal to output nodes
        }

        out_layer = multilayer_perceptron(tf_train_dataset, weights, biases)

        L = tf.nn.softmax_cross_entropy_with_logits(out_layer, tf_train_labels)
        L += regularization(weights)
        loss = tf.reduce_mean(L)

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(out_layer)
        valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights, biases))
        test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights, biases))

In [79]:
num_steps = 3001


# train this nn total is @param num_steps. 
with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2514.304688
Minibatch accuracy: 15.6%
Validation accuracy: 26.0%
Minibatch loss at step 500: 65.365730
Minibatch accuracy: 85.9%
Validation accuracy: 84.4%
Minibatch loss at step 1000: 2.665214
Minibatch accuracy: 84.4%
Validation accuracy: 84.4%
Minibatch loss at step 1500: 0.705578
Minibatch accuracy: 85.9%
Validation accuracy: 84.6%
Minibatch loss at step 2000: 0.753497
Minibatch accuracy: 79.7%
Validation accuracy: 84.5%
Minibatch loss at step 2500: 0.780378
Minibatch accuracy: 80.5%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 0.795823
Minibatch accuracy: 81.2%
Validation accuracy: 84.3%
Test accuracy: 90.6%


> Result Analyse: The Test accuracy and validation accuracy are higher compared to no-regularization. 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [80]:
def multilayer_perceptron(x, weights, biases):
     # Training computation.
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1_output = tf.nn.relu(layer_1)
    ## change data to relu
    
    logits = tf.matmul(layer_1_output, weights['h2']) + biases['b2']
    
    return logits


def regularization(weights):
    alpha = 0.007
    return alpha * sum(map(lambda w_i: tf.nn.l2_loss(w_i), weights.values()))


batch_size = 10

    # initial neoral network by train size @param batch_size
n_hidden = 1024
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
    
        weights = {
            'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden])),
            'h2': tf.Variable(tf.truncated_normal([n_hidden, num_labels])),
            # output is num_labels
        }

        biases = {
            'b1': tf.Variable(tf.zeros([n_hidden])), # size equal to output nodess
            'b2': tf.Variable(tf.zeros([num_labels])), # size equal to output nodes
        }

        out_layer = multilayer_perceptron(tf_train_dataset, weights, biases)

        L = tf.nn.softmax_cross_entropy_with_logits(out_layer, tf_train_labels)
        L += regularization(weights)
        loss = tf.reduce_mean(L)

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(out_layer)
        valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights, biases))
        test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights, biases))
        

num_steps = 3001
# train this nn total is @param num_steps. 
with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2441.838135
Minibatch accuracy: 20.0%
Validation accuracy: 18.5%
Minibatch loss at step 500: 396.821747
Minibatch accuracy: 60.0%
Validation accuracy: 59.6%
Minibatch loss at step 1000: 12.047380
Minibatch accuracy: 80.0%
Validation accuracy: 69.9%
Minibatch loss at step 1500: 2.035784
Minibatch accuracy: 70.0%
Validation accuracy: 62.1%
Minibatch loss at step 2000: 0.826251
Minibatch accuracy: 90.0%
Validation accuracy: 66.5%
Minibatch loss at step 2500: 2.573372
Minibatch accuracy: 50.0%
Validation accuracy: 60.7%
Minibatch loss at step 3000: 1.161802
Minibatch accuracy: 70.0%
Validation accuracy: 63.4%
Test accuracy: 70.0%


> Result Analyse: Batch accuracy is still well, and its speed become quicker. But the validation accuracy is lower much more, and test accuracy also is lower much more. 

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
